# Libraries

In [1]:
# pip install mtcnn
# pip install cmake
# pip intall dlib
# and vs studio

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import time
import dlib
from mtcnn import MTCNN
from retinaface import RetinaFace

# Loading Models

In [2]:
model_mtcnn = MTCNN()

model_dlibHOG = dlib.get_frontal_face_detector()

model_dlibCNN = dlib.cnn_face_detection_model_v1("../Models/mmod_human_face_detector.dat")

modelFile = "../Models/res10_300x300_ssd_iter_140000.caffemodel"
configFile = "../Models/Resnet_SSD_deploy.prototxt"
model_SSD = cv2.dnn.readNetFromCaffe(configFile, modelFile)

#retina-face needs no initial setup

classifier = cv2.CascadeClassifier('../Models/haarcascade_frontalface2.xml')

# Model Functions

In [44]:
def mtcnn(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    faces = model_mtcnn.detect_faces(img_rgb)
    for result in faces:
        x, y, w, h = result['box']
        x1, y1 = x + w, y + h
        cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)


def dlibHog(img):
    faces = model_dlibHOG(img, 2)
    for result in faces:
        x = result.left()
        y = result.top()
        x1 = result.right()
        y1 = result.bottom()
        cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
    
    
# def dlib_cnn(img):
#     faces = model_dlibCNN(img, 2)
#     for result in faces:
#         x = result.left()
#         y = result.top()
#         x1 = result.right()
#         y1 = result.bottom()
#         cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)


def ssd(img):
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 1.0, (300, 300), (104.0, 117.0, 123.0))
    model_SSD.setInput(blob)
    faces = model_SSD.forward()
    for i in range(faces.shape[2]):
        confidence = faces[0, 0, i, 2]
        if confidence > 0.5:
            box = faces[0, 0, i, 3:7] * np.array([width, height, width, height])
            (x, y, x1, y1) = box.astype("int")
            cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)
    
    
def retinaface(img):
    faces = RetinaFace.detect_faces(img)
    try:
        for i in faces.keys():
            facial_parts = faces[i]
            recognize_face_area = facial_parts["facial_area"]
            cv2.rectangle(img, (recognize_face_area[2], recognize_face_area[3]), (recognize_face_area[0], recognize_face_area[1]), (0, 0, 255), 2)
    except:
        pass
    
    
def haar(img):
    faces = classifier.detectMultiScale(img)
    for result in faces:
        x, y, w, h = result
        x1, y1 = x + w, y + h
        cv2.rectangle(img, (x, y), (x1, y1), (0, 0, 255), 2)

# Image Processing with timestamps


In [45]:
output_path = 'Results\Light\Outputs\SSD'

if not os.path.exists(output_path):
    os.makedirs(output_path)


input_path = 'Results\Light\Inputs'
images = os.listdir(input_path)

for image in images:
    img = cv2.imread(os.path.join(input_path, image))
    start = time.time()
    ssd(img)
    end = time.time()
    total_t = end - start
    cv2.putText(img, f'Elapsed Time: {format(float(total_t), ".3f")}', (5,25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imwrite(os.path.join(output_path, image), img)
    plt.figure(figsize= (10, 10))
    plt.imshow(img[:, :, ::-1])
    plt.show()

# Video Processing with FPS & Timestamp

In [ ]:
input_path = 'Results\drone.mp4'

video = cv2.VideoCapture(input_path)


width = int(video.get(3))
height = int(video.get(4))

# width_dis = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
# height_dis = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Create a window to display the video
cv2.namedWindow("Video", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Video", width, height)

fourcc = cv2.VideoWriter_fourcc(*'MP4V')

# Define the codec and create VideoWriter object.The output is stored in 'output.mp4' file.
out = cv2.VideoWriter('Results\drone_MTCNN.mp4', fourcc, 24, (width,height))

elapsed = 0
if (video.isOpened() == False):
    print("Web Camera not detected")
while (True):
    ret, frame = video.read()
    if ret == True:
        start = time.time()
        #for faster times use: ssd or haar
        mtcnn(frame)
        end = time.time()
        totalTime = end - start
        elapsed += totalTime
        fps = 1 / totalTime
        cv2.putText(frame, f'FPS: {int(fps)}, Elapsed Time: {format(float(elapsed), ".3f")}', (5,40), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
        out.write(frame)
        cv2.imshow("Video",frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

video.release()
out.release()
cv2.destroyAllWindows()